작성자: https://github.com/sawoo9410 <br>
최초 작성일자: 2023-06-09 <br>
최종 작성일자: - <br>

### Ref.
- 데이터셋 출처: Smilegate AI(https://github.com/smilegate-ai/korean_unsmile_dataset)
- baseline 참조: https://colab.research.google.com/drive/1NKYYVSex__vde-lnYCmsRmyHjJhV6cKt?usp=sharing

----

##### 해당 코드는 Smilegate AI 센터에서 공개한 한국어 혐오표현 "UnSmile" 데이터셋을 탐색한 코드입니다.
##### baseline 코드를 참조해서 코드 내용을 자세히 서술하거나, 탐색합니다.

----

### 0. 결과해석 후 개선점 추론

원래 탐색적 자료 분석(EDA)는 데이터를 분석하기 전에 시행하는 것이지만, 해당 EDA 코드는 baseline을 통해 결과를 도출 후 모델 이외 즉, 데이터 부문에서 수정할만한 사항들을 탐구합니다.

#### baseline 결과 해석

- 가장 클래스의 비율이 적었던 "연령"과 "기타 혐오"를 확인했을 때, f1-score 기준 "연령"의 경우 좋은 결과를 보였으나, "기타 혐오"의 경우 모든 클래스 중 가장 낮은 결과를 보임
- "악플/욕설"의 경우 "clean"을 제외하고 가장 많은 개수를 가진 클래스지만, f1-score 기준 "기타 혐오" 다음으로 낮은 결과를 보임
- 정확도는 약 0.73로 아쉬운 결과.

{'여성/가족': 0,
 '남성': 1,
 '성소수자': 2,
 '인종/국적': 3,
 '연령': 4,
 '지역': 5,
 '종교': 6,
 '기타 혐오': 7,
 '악플/욕설': 8,
 'clean': 9}

### 1. 데이터셋 불러오기

In [1]:
import pandas as pd

train_df = pd.read_csv("./korean_unsmile_dataset-main/unsmile_train_v1.0.tsv", delimiter='\t')
valid_df = pd.read_csv("./korean_unsmile_dataset-main/unsmile_valid_v1.0.tsv", delimiter='\t')

train_df.shape, valid_df.shape

((15005, 12), (3737, 12))

In [2]:
train_df.head() 

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


In [3]:
valid_df.head() 

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,0,1,0,0,0,0,0,0,0,0,0
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1,0,0,0,0,0,0,0,0,0,0
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,0,0,0,1,0,0,1,0,0,0,0
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,0,1,0,0,0,0,0,0,0,0,0
4,바지 내리다 한남들 와꾸 보고 올려뿟노,0,1,0,0,0,0,0,0,0,0,0


In [6]:
unsmile_labels = ["여성/가족","남성","성소수자","인종/국적","연령","지역","종교","기타 혐오","악플/욕설","clean"]

##### class imbalance 확인

train, valid 데이터 셋의 클래스 불균형을 확인<br>

- 총 10개의 클래스, 각 클래스 별 비율이 0.1이 가장 이상적
- '연령'과 '기타 혐오'가 가장 낮은 비율
- 나머지 클래스는 0.1에 근사함

In [7]:
dict_train_balance = {}
dict_valid_balance = {}

def class_balance(dict_, labels_name, class_n):
    ratio = []
    
    for i in labels_name:
        dict_[i] = class_n[i]
        ratio.append(round(class_n[i]/class_n.sum(),2))
    
    return dict_, ratio

dict_train_balance, train_ratio = class_balance(dict_train_balance, unsmile_labels, train_df.iloc[:, 1:11].sum())
dict_valid_balance, valid_ratio = class_balance(dict_valid_balance, unsmile_labels, valid_df.iloc[:, 1:11].sum())
print(dict_train_balance, '\n', train_ratio)
print(dict_valid_balance, '\n', valid_ratio)

{'여성/가족': 1599, '남성': 1347, '성소수자': 1141, '인종/국적': 1728, '연령': 603, '지역': 1052, '종교': 1181, '기타 혐오': 569, '악플/욕설': 3143, 'clean': 3739} 
 [0.1, 0.08, 0.07, 0.11, 0.04, 0.07, 0.07, 0.04, 0.2, 0.23]
{'여성/가족': 394, '남성': 334, '성소수자': 280, '인종/국적': 426, '연령': 146, '지역': 260, '종교': 290, '기타 혐오': 134, '악플/욕설': 786, 'clean': 935} 
 [0.1, 0.08, 0.07, 0.11, 0.04, 0.07, 0.07, 0.03, 0.2, 0.23]


### 2.  EDA

---

#### 추론 1. 왜, "악플/욕설" 클래스는 다른 클래스에 비해 데이터 양이 많음에도 불구하고 낮은 f1-score를 기록했을까?
- 예상 답 1. UnSmile 데이터 셋은 multi-label classfication 문제로, "악플/욕설" 클래스가 다른 클래스와 많이 중복으로 라벨링 되어 있어 분류하기 어려웠다.
- 예상 답 2. "악플/욕설"이 너무 다양해서 분류하기 어려웠다.

In [33]:
train_df_y = train_df.iloc[:, 1:11]
train_df_y_none_cl = train_df_y.iloc[:, :-1]

train_df_y_none_cl['중복된 혐오 표현 개수'] = train_df_y_none_cl.sum(axis=1)

# 혐오 표현이 2개 이상 사용된 행
twiceUp_bad_words = train_df_y_none_cl[train_df_y_none_cl['중복된 혐오 표현 개수'] >= 2]

In [34]:
twiceUp_bad_words[twice_bad_words['악플/욕설'] == 1]

,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,중복된 혐오 표현 개수


In [36]:
twiceUp_bad_words.sum()

여성/가족            308
남성               219
성소수자             187
인종/국적            469
연령               158
지역               197
종교               317
기타 혐오            238
악플/욕설              0
중복된 혐오 표현 개수    2093
dtype: int64

오히려 "악플/욕설"의 경우, 중복 키워드가 존재하지 않는다.<br>

특이하게 악플/욕설이 게임 상에서 보편적으로 사용되는 혐오 표현이며 보통 다른 종류의 혐오 표현과 같이 사용될 것 같은데, 아예 0이라는 수치가 이해가 잘 가지 않는다.

In [40]:
train_df[train_df['악플/욕설']==1]

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
7,나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임,0,0,0,0,0,0,0,0,1,0,0
17,위로와 격려를 호텔에서 해? 왜? 저여자도 정상은 아니구만 완전 지네 할아버지보다 ...,0,0,0,0,0,0,0,0,1,0,0
19,띄울사람이 진짜 없긴 없나보다,0,0,0,0,0,0,0,0,1,0,0
22,쑈 빼고 아무것도 없는 개돼지 정권이노 탄핵 개힘조,0,0,0,0,0,0,0,0,1,0,0
27,뭐하냐 기업이네 기업이여 기업에 돈갖다 받치는 어이없는 시키들도 문제다,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14978,화장이나 지우고 얘기해!!! 아니면 티 안나게 잘하던지!!,0,0,0,0,0,0,0,0,1,0,0
14992,내전 나겠네 ㅋㅋ 팝콘 사놔라,0,0,0,0,0,0,0,0,1,0,0
14993,옷다벗겨서 북송해라ㅋㅋㅋㅋㅋㅋ,0,0,0,0,0,0,0,0,1,0,0
14994,어릴때부터 북한과 빨치산에 세뇌된 게 무섭다.아주 무서운 열등감과 피해의식을 갖고 ...,0,0,0,0,0,0,0,0,1,0,0


7번 행 "나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임"의 문장의 경우 "연령" 부문과 충분히 중복되어 표기할만한 것 같다.<br>
17번 행 "위로와 격려를 호텔에서 해? 왜? 저여자도 정상은 아니구만 완전 지네 할아버지보다 ..."의 문장도 "여성/가족" 부문과 충분히 중복되어 표기할만한 것 같다.<br>
22번 행 "쑈 빼고 아무것도 없는 개돼지 정권이노 탄핵 개힘조"의 문장도 "기타 혐오" 부문에 충분히 중복되어 표기할만한 것 같다.<br>
14994번 행 "어릴때부터 북한과 빨치산에 세뇌된 게 무섭다.아주 무서운 열등감과 피해의식을 갖고 ...	"의 문장도 "연령" 부문과 충분히 중복되어 표기할만한 것 같다.

이해가 가질 않아 혐오 분류의 기준을 재확인하기 위해 데이터셋 공식 GIthub에서 가져왔다.

[여성/가족]
- 여성성 및 여성의 성역할에 대한 통념을 고착시키는 발언, 여성 차별을 희화화하는 발언, 페미니즘·여성가족부 전반에 대한 악플 등을 포함합니다.
- 간호사, 여경 등 여성을 중심으로 구성된 집단에 대한 조롱 역시 여성 혐오 표현으로 분류하였습니다.
- 비혼주의자, 미혼모, 동성 부부 등 전통적이지 않은 형식의 가족에 대한 혐오 발언 역시 본 카테고리에 포함됩니다.

[남성]
- 집단으로서의 남성 일반을 비하, 조롱, 희화화하는 발언들입니다.

[성소수자]
- 성소수자(레즈비언, 게이, 바이섹슈얼, 트랜스젠더 등)를 배척하는 발언입니다.
- 이성애 이외의 섹슈얼리티를 부정적으로 묘사하거나 성소수자를 희화화하는 표현들을 포함합니다.

[인종/국적]
- 특정 인종(흑인, 아시안 등)과 국적(일본인, 아프가니스탄인, 베트남인 등)에 대한 욕설, 고정관념, 조롱을 다룹니다.
- 종교·인종·국가에 대해 암묵적으로 함께 지칭하는 소재 (e.g. 무슬림, 난민)의 발언들 역시 본 카테고리에 포함됩니다.

[연령]
- 특정 세대나 연령을 비하하는 은어의 사용 및 혐오 표현을 분류하였습니다.

[지역]
- 특정 지역에 대한 은어 및 혐오 표현을 분류하였습니다.

[종교]
- 특정 종교에 대한 혐오 및 종교인 집단에 대한 비난을 분류하였습니다.

[기타혐오]
- 위에서 정의한 카테고리 이외의 집단을 대상으로 하는 혐오 표현을 분류하였습니다. (e.g. 장애인, 정부, 기자, 경찰, 차별금지법 반대 등)

[악플/욕설]
- 어떤 집단을 향한 혐오 표현인지 지칭할 수는 없지만, 타인 혹은 외모에 대한 비하/욕설이 포함되어 있거나, 불쾌감을 주거나, 악플과 음란성 문장을 분류하였습니다.

[Clean]
- 혐오표현, 욕설, 불쾌감, 음란성 내용을 포함하고 있지 않은 일반 문장을 분류하였습니다.

혐오 분류 기준을 보니, **앞의 8개의 클래스로 명확히 특정할 수 없는 경우 "악플/욕설"로 분류하는 것 같다.** 즉, 속된 말로 혐오표현을 짬처리하는,,, 클래스로 추정된다.

이러다 보니, "악플/욕설"이 모델 결과(f1-score 기준)가 낮을 수 밖에 없는 것 같다. 또한 **앞의 8개의 클래스로 명확히 특정할 수 없는 경우 "악플/욕설"로 분류가 되는게 맞다면 중복 혐오 표현 개수가 존재하지 않는 것이 이해가 간다.**

---

#### 추론 2. 왜, "연령" 클래스는 다른 클래스에 비해 데이터 양이 적음에도 불구하고 높은 f1-score를 기록했으며, "기타 혐오" 클래스는 낮은 f1-score를 기록했을까?
- 예상 답. 연령이란 키워드는 확실하게 분류되지만, 기타 혐오란 키워드는 말 그대로 기타 속성을 가지기 때문에 명확히 분류되기 어려운 내용일 것이다.